In [1]:
import geopandas as gpd
import pandas as pd

# Load all Danish postal‐code areas (GeoJSON) directly from DAWA
postal_gdf = gpd.read_file(
    "https://dawa.aws.dk/postnumre?format=geojson"
).to_crs(epsg=4326)  # ensure WGS84 lon/lat

In [2]:
postal_gdf.shape

(1089, 10)

In [3]:
postal_gdf.columns

Index(['nr', 'navn', 'stormodtager', 'visueltcenter_x', 'visueltcenter_y',
       'ændret', 'geo_ændret', 'geo_version', 'dagi_id', 'geometry'],
      dtype='object')

In [4]:
# Inspect columns: 
#   'nr' is the postal code (as integer), 'navn' the name, and 'geometry' the polygon
print(postal_gdf[['nr','navn']].head())

     nr         navn
0  1050  København K
1  1051  København K
2  1052  København K
3  1053  København K
4  1054  København K


In [5]:
# Compute true geometric centroids (lat/lon) for each postal area
postal_gdf["zip_centroid"] = postal_gdf.geometry.centroid
postal_gdf["zip_lon"] = postal_gdf.zip_centroid.x
postal_gdf["zip_lat"] = postal_gdf.zip_centroid.y


/tmp/ipykernel_12237/3923013708.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  postal_gdf["zip_centroid"] = postal_gdf.geometry.centroid


In [6]:
# Rename and keep only the columns we need
postal_gdf = postal_gdf.rename(columns={"nr":"postal_code"})[[
    "postal_code", "zip_lat", "zip_lon", "geometry"
]]

In [7]:
# Load your population‐by‐zip CSV
pop = pd.read_csv("population_df.csv", dtype={"postal_code": str})
# ensure the codes match type:
pop["postal_code"] = pop["postal_code"].str.zfill(4)

In [8]:
# Merge the pop stats onto the postal GeoDataFrame
postal_with_pop = postal_gdf.merge(
    pop,
    on="postal_code",
    how="left"
)

In [9]:
postal_with_pop

,postal_code,zip_lat,zip_lon,geometry,neighborhood_code,neighborhood_name,postal_area,postal_code_m2,Total,Men,Women,population_density_km2,restaurant_count
0,1050,55.680407,12.585996,"MULTIPOLYGON (((12.58454 55.68079, 12.58457 55...",101.0,København,København K,39053.0,20.0,10.0,10.0,512.124549,16.0
1,1051,55.679881,12.590506,"MULTIPOLYGON (((12.58929 55.68077, 12.58936 55...",101.0,København,København K,55643.0,399.0,191.0,208.0,7170.713297,28.0
2,1052,55.678842,12.590405,"MULTIPOLYGON (((12.59077 55.67866, 12.59077 55...",101.0,København,København K,6563.0,421.0,204.0,217.0,64147.493524,2.0
3,1053,55.678042,12.590729,"MULTIPOLYGON (((12.59034 55.67841, 12.59036 55...",101.0,København,København K,2434.0,404.0,181.0,223.0,165981.922761,2.0
4,1054,55.678055,12.588838,"MULTIPOLYGON (((12.58873 55.67804, 12.58875 55...",101.0,København,København K,4953.0,392.0,201.0,191.0,79143.953160,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,9940,57.238812,11.130990,"MULTIPOLYGON (((11.6889 57.08614, 11.39651 57....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1085,9970,57.512627,10.522827,"MULTIPOLYGON (((10.56671 57.50988, 10.57191 57...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1086,9981,57.541614,10.458983,"MULTIPOLYGON (((10.40957 57.5689, 10.4095 57.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1087,9982,57.640310,10.336764,"MULTIPOLYGON (((10.40957 57.5689, 10.4097 57.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Turn your pedestrian CSV into a GeoDataFrame
ped = pd.read_csv("expanded_pedestrian_traffic_interpolated.csv")
ped_gdf = gpd.GeoDataFrame(
    ped,
    geometry=gpd.points_from_xy(ped.longitude, ped.latitude),
    crs="EPSG:4326"
)

In [11]:
# Spatial-join: tag each point with its containing postal polygon’s stats
joined = gpd.sjoin(
    ped_gdf,
    postal_with_pop,
    how="left",
    predicate="within"
)


In [12]:
result = joined[[
    "latitude","longitude","aadt_fod_7_19","hvdt_fod_7_19",
    "postal_code",
    "Total","Men","Women","population_density_km2"
]]
result.to_csv("pedestrian_with_zip_stats.csv", index=False)

In [13]:
result

,latitude,longitude,aadt_fod_7_19,hvdt_fod_7_19,postal_code,Total,Men,Women,population_density_km2
0,55.707340,12.454837,1305.544721,1333.896656,2700,45950.0,22892.0,23058.0,5603.658537
1,55.708353,12.454837,1207.564470,1232.454639,2700,45950.0,22892.0,23058.0,5603.658537
2,55.709365,12.454837,1095.344012,1116.321090,2700,45950.0,22892.0,23058.0,5603.658537
3,55.710377,12.454837,994.907609,1012.386543,2700,45950.0,22892.0,23058.0,5603.658537
4,55.711389,12.454837,936.447968,951.898583,2700,45950.0,22892.0,23058.0,5603.658537
...,...,...,...,...,...,...,...,...,...
7057,55.720497,12.665042,3859.789405,3988.530266,1433,0.0,0.0,0.0,0.000000
7058,55.719485,12.666839,3881.197561,4010.677412,1433,0.0,0.0,0.0,0.000000
7059,55.720497,12.666839,3868.851499,3997.897448,1433,0.0,0.0,0.0,0.000000
7060,55.703291,12.731518,4049.968794,4185.249819,1433,0.0,0.0,0.0,0.000000
